In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
from PortfolioToolboxQP.PortfolioToolboxQP.Qp_Solver import qp_solver_cpp as solve

Il faut remplir le code suivant avec ce qu'on a calculé dans les autres fichiers. Faut juste faire attention à ce qu'on est les mêmes colonnes pour tous sinon on va avoir des problèmes de dimensions.

In [ ]:
datamsci = pd.read_csv("data/DataMSCI.csv", sep=";")
weights = datamsci[["ISSUER_ISIN", "MarketCap_USD"]]
benchmark = weights #drop the missing columns
benchmark["MarketCap_USD"] = benchmark["MarketCap_USD"] / benchmark["MarketCap_USD"].sum()
b = benchmark["MarketCap_USD"].values

In [ ]:
sigma = covariance
CI = carbon_intensity
CI0 = carbon intensity 2023
R = 0.35 #reduction_rate cf net_zero_emissions for other values

J'ai pris le problème le plus simple avec le moins de contrainte (min 1/2(x-b)Tsigma(x-b) avec comme contraintes la carbone intensity, et le bail C0 des long only constraints [x entre 0 et 1 et qui se somme à 1])

In [ ]:
y = solve(Q=sigma, p=None, G=CI, h=(1-R)*CI0@b - CI@b, A=np.ones(b.shape).T, b=-np.ones(b.shape).T@b+1, lb=np.zeros(b.shape)-b, ub=np.ones(b.shape)-b)

""" x is what we want : here the relative weights on the differents firms. The following Y is what we call "x-b". 
 We have to add the benchmark at the end

                            minimize 0.5*Y.T@Q@Y + p@Y under constraints AY = B and GY <= h and  lb <= Y <= ub
    
    all parameters are ndarray

    Q: sigma for us
    p: 0

    CI*(X=Y+b) <= (1-R)CI_0*b donc CI*Y <= (1-R)*CI_0*b - CI*b
    G: Matrix of inequalty constraints → CI
    h: Vector of inequality constraints → -R*CI*b

    1.T(Y+b)=1 donc 1TY = - 1Tb + 1
    A: Matrix of equality constraints → 1T
    b: Vector of equality constraints → -1Tb + 1
    
    lb: Lower bound for Y : for x : 0
    ub: Upper bound for Y : for x : 1

    return: optimal vector if found : Y=X-b
    """

x = y+b 
print(x)
plt.plot(np.linspace(1,len(x)),x)
tracking_error = 0.5*y.T@sigma@y